In [1]:
import os
import re
import pandas as pd
from nltk.tokenize import RegexpTokenizer, sent_tokenize
import numpy as np
import edgar10kmda.edgar as ed

In [2]:
df1=pd.read_csv('/Volumes/Elements/MasterThesis/Thesis/DataSheets/FraudCases.csv',header=None, sep=',')
df2=pd.read_csv('/Volumes/Elements/MasterThesis/Thesis/DataSheets/index.10k.csv',header=None, sep=',')



In [3]:
#Full Fraud Cases(identified from AAER) manually from Meirut
#to be scraped from SEC 
df1.head()

,0,1,2,3,4,5,6,7,8,9
0,Uni-Pixel 2013,https://www.sec.gov/Archives/edgar/data/117101...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Uni-Pixel 2014,https://www.sec.gov/Archives/edgar/data/117101...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,om group inc 2002,https://www.sec.gov/Archives/edgar/data/899723...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,om group inc 2003,https://www.sec.gov/Archives/edgar/data/899723...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,SMF Energy Corp 2011,https://www.sec.gov/Archives/edgar/data/102445...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
#Full Index10k Dataframe from 1993-2019 with shape (187869, 5)
df2.head()

,0,1,2,3,4
0,10-K,AMERICAN MEDICAL HOLDINGS INC,861439,1993-11-29,edgar/data/861439/0000912057-94-000263.txt
1,10-K,DISNEY WALT CO,29082,1993-12-22,edgar/data/29082/0000950131-94-000021.txt
2,10-K,ELIZABETHTOWN GAS CO,32377,1993-12-13,edgar/data/32377/0000032377-94-000001.txt
3,10-K,INTERNATIONAL GAME TECHNOLOGY,353944,1993-12-23,edgar/data/353944/0000353944-94-000005.txt
4,10-K,ABBOTT LABORATORIES,1800,1994-03-03,edgar/data/1800/0000912057-94-000771.txt


In [7]:
FraudurlList=df1[1].tolist()
FileNameList=[k.split('Archives/', 1)[1] for k in FraudurlList]
df1[2]=pd.Series(FileNameList)
FraudCompanyNameYearList=df1[0].tolist()
FraudCompanyNameList=[k[:-5] for k in FraudCompanyNameYearList]
df1[0]=pd.Series(FraudCompanyNameList)
FraudCompanyYearList=[k[-4:] for k in FraudCompanyNameYearList]
df1[1]=pd.Series(FraudCompanyYearList)


In [8]:
df1.head()

,0,1,2,3,4,5,6,7,8,9
0,Uni-Pixel,2013,edgar/data/1171012/0001185185-13-000386.txt,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Uni-Pixel,2014,edgar/data/1171012/0001185185-14-000424.txt,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,om group inc,2002,edgar/data/899723/0000950152-02-002583.txt,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,om group inc,2003,edgar/data/899723/0000950152-03-003357.txt,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,SMF Energy Corp,2011,edgar/data/1024452/0001144204-11-055366.txt,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
#Sub DataFrame of DF2 with just the Fraud Cases from DF1 has a shape of 654 Fraud Cases
df3=df2.loc[df2[4].isin(FileNameList)]
df3.to_csv('FraudCases9319.csv', header=False, index=False)
df3.head()

,0,1,2,3,4
1489,10-K,POLICY MANAGEMENT SYSTEMS CORP,356226,1994-04-13,edgar/data/356226/0000356226-94-000007.txt
3025,10-K,STRUCTURAL DYNAMICS RESEARCH CORP /OH/,820235,1995-03-28,edgar/data/820235/0000906318-95-000015.txt
6292,10-K,CUC INTERNATIONAL INC /DE/,723612,1996-04-25,edgar/data/723612/0000950112-96-001227.txt
10272,10-K,HEALTHSOUTH CORP,785161,1997-03-27,edgar/data/785161/0001005150-97-000195.txt
13016,10-K,CUC INTERNATIONAL INC /DE/,723612,1997-05-01,edgar/data/723612/0001005477-97-001218.txt


In [202]:
Df3urlList=df3[4].tolist()
len(Df3urlList)

654

In [11]:
#from 662 Fraud Cases manually retrieved from Meirut, 8 could not be found in Index File csv
df3.shape

(654, 5)

In [12]:
len(FileNameList)

691

In [13]:
len(FileNameList)
#remove Duplicates from a Python List--means in DF1 tehre are 691-662 duplicates. 
#total fraud cases 662
FileNameList2=list(dict.fromkeys(FileNameList))
len(FileNameList2)

662

In [14]:
def Diff(li1, li2): 
    return (list(set(li1) - set(li2)))

In [15]:
NotFoundList=Diff(FileNameList,Df3urlList )
NotFoundList

['edgar/data/355356/0001193125-11-276829.txt',
 'edgar/data/914712/0001193125-11-271739.txt',
 'edgar/data/1110783/0000950123-11-098240.txt',
 'edgar/data/40570/0001140361-11-059150.txt',
 'edgar/data/880446/0001047469-11-009945.txt',
 'edgar/data/1175680/0001193125-11-295429.txt',
 'edgar/data/857005/0001193125-11-320097.txt',
 'edgar/data/913760/0000913760-11-000012.txt']

In [16]:
len(NotFoundList)

8

In [ ]:
#secfname are unique!! single identifier based on which we can merge all othe rcolumns

In [47]:
duplicateRowsdf3=df3[df3.duplicated([4])]

In [48]:
print("Duplicate Rows except first occurrence based on all columns are :")
print(duplicateRowsdf3)

Duplicate Rows except first occurrence based on all columns are :
Empty DataFrame
Columns: [0, 1, 2, 3, 4]
Index: []


In [17]:
df2.shape

(187869, 5)

In [18]:
#additional Fraud DataSet
df4=pd.read_csv('/Volumes/Elements/MasterThesis/Thesis/DataSheets/FraudCases2.csv', sep=',')

In [19]:
df1=df1.rename(columns={0:"Company", 1:"Year", 2:"url"})
df1.head()

,Company,Year,url,3,4,5,6,7,8,9
0,Uni-Pixel,2013,edgar/data/1171012/0001185185-13-000386.txt,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Uni-Pixel,2014,edgar/data/1171012/0001185185-14-000424.txt,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,om group inc,2002,edgar/data/899723/0000950152-02-002583.txt,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,om group inc,2003,edgar/data/899723/0000950152-03-003357.txt,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,SMF Energy Corp,2011,edgar/data/1024452/0001144204-11-055366.txt,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
df4.head()
col_list=['Company','Year','Filename']

In [21]:
df4.shape

(403, 3)

In [22]:
df4=df4.reindex(columns=col_list)

In [23]:
df4.head()

,Company,Year,Filename
0,Adelphia,1999,Adelphia_1999_10K.txt
1,Adelphia,2000,Adelphia_2000_10K.txt
2,Advanced marketing services,2001,AdvancedMarketingServices_2001_10K.txt
3,Advanced marketing services,2002,AdvancedMarketingServices_2002_10K.txt
4,Advanced marketing services,2003,AdvancedMarketingServices_2003_10K.txt


In [ ]:
def extract_secfname(path):
    html_regex = re.compile(r'<.*?>')
    extracted_data=[]
    for fileName in os.listdir(path):
            filenameopen = os.path.join(path, fileName)
            #print(filenameopen)
            #print(fileName)
            resultdict = dict()
            with open(filenameopen, 'r', encoding='utf-8', errors="replace") as in_file:
                    content = in_file.read()
                    content = re.sub(html_regex,'',content)
                    content = content.replace('&nbsp;','')
                    content = re.sub(r'&#\d+;', '', content)
                    
                    matches_CIK_regex = re.findall("CENTRAL INDEX KEY:\t\t.*?^\n*", content, re.IGNORECASE | re.DOTALL | re.MULTILINE)
                    if matches_CIK_regex:
                        #print(matches_secfname )
                        result = max(matches_CIK_regex, key=len)
                        result = str(result).replace('\n', '')
                        result = str(result).replace('\t', '')
                        result = str(result).replace('CENTRAL INDEX KEY:', '')
                        result =str(int(str(result)))
                        resultdict['CIK'] = result
                    else:
                         resultdict['CIK'] = ""
                    
                    matches_coname_regex = re.findall("COMPANY CONFORMED NAME:\t\t.*?^\n*", content, re.IGNORECASE | re.DOTALL | re.MULTILINE)
                    if matches_coname_regex:
                        #print(matches_secfname )
                        result = max(matches_coname_regex, key=len)
                        result = str(result).replace('\n', '')
                        result = str(result).replace('\t', '')
                        result = str(result).replace('COMPANY CONFORMED NAME:', '')
                        resultdict['CONAME'] = result
                    else:
                         resultdict['CONAME'] = ""
                            
                    matches_form = re.findall("FORM TYPE:\t\t.*?^\n*", content, re.IGNORECASE | re.DOTALL | re.MULTILINE)
                    if matches_form:
                        #print(matches_secfname )
                        result = max(matches_form, key=len)
                        result = str(result).replace('\n', '')
                        result = str(result).replace('\t', '')
                        result = str(result).replace('FORM TYPE:', '')
                        resultdict['FORM'] = result
                    else:
                         resultdict['FORM'] = ""
        
                    matches_accnbr = re.findall("ACCESSION NUMBER:\t\t.*?^\n*", content, re.IGNORECASE | re.DOTALL | re.MULTILINE)
                    if matches_accnbr:
                        #print(matches_secfname )
                        result = max(matches_accnbr, key=len)
                        result = str(result).replace('\n', '')
                        result = str(result).replace('\t', '')
                        result = str(result).replace('ACCESSION NUMBER:', '')
                        #contrsucting SECFNAME from CIK and accnbr stored in result
                        resultdict['SECFNAME'] = "edgar/data/"+resultdict['CIK']+"/"+ result +".txt"
                    else:
                         resultdict['SECFNAME'] = ""
                                
                    
                    extracted_data.append(resultdict)
                
            in_file.close()
    return extracted_data

In [ ]:
#Extract SEC Filename from the whole Fraud Cases FOlder 
path4="/Volumes/Elements/MasterThesis/sec/Fraud2"
DictList2=extract_secfname(path4)
#List of all SECFnames
SECFNAME_list=[d["SECFNAME"] for d in DictList2]
print(len(DictList2))

In [136]:
len(SECFNAME_list)

403

In [128]:
FORM10k_list=[d["FORM"] for d in DictList2]

In [130]:
FORM10k_list=[d for d in DictList2 if d["FORM"]=="10-K" ]
print(len(FORM10k_list))
FORM10ksb_list=[d for d in DictList2 if d["FORM"]=="10KSB" ]
print(len(FORM10ksb_list))
FORM10k405_list=[d for d in DictList2 if d["FORM"]=="10-K405" ]
print(len(FORM10k405_list))
FORM20F_list=[d for d in DictList2 if d["FORM"]=="20-F" ]
print(len(FORM20F_list))

256
40
50
23


In [106]:
#Fraud2 Cases 
print(len(DictList2))

403


In [107]:
dfFraud2 = pd.DataFrame(DictList2)
dfFraud2.head()

,CIK,CONAME,FORM,SECFNAME
0,5272,AMERICAN INTERNATIONAL GROUP INC,10-K,edgar/data/5272/0000950123-01-002952.txt
1,5272,AMERICAN INTERNATIONAL GROUP INC,10-K,edgar/data/5272/0000950123-02-003222.txt
2,796486,ADELPHIA COMMUNICATIONS CORP,10-K,edgar/data/796486/0000796486-00-000008.txt
3,796486,ADELPHIA COMMUNICATIONS CORP,10-K,edgar/data/796486/0000796486-01-000012.txt
4,814580,ADVANCED MARKETING SERVICES INC,10-K,edgar/data/814580/0001095811-01-503019.txt


In [192]:
dfFraud2.groupby("FORM").count()
#33 Fraud Cases in Fraud2 Folder are empty rows, meaning there are no 10k or 20F Files

,CIK,CONAME,SECFNAME
FORM,,,
,33,33,33
10-K,256,256,256
10-K405,50,50,50
10KSB,40,40,40
10KSB40,1,1,1
20-F,23,23,23


In [111]:
dfFraud2.shape

(403, 4)

In [159]:
dfFraud2_10k=dfFraud2[dfFraud2['FORM']=="10-K"]

In [160]:
dfFraud2_10k.shape

(256, 4)

In [161]:
SECFnameFraud2_10kList=dfFraud2_10k["SECFNAME"].tolist()

In [117]:
DictList2[1]

{'CIK': '5272',
 'CONAME': 'AMERICAN INTERNATIONAL GROUP INC',
 'FORM': '10-K',
 'SECFNAME': 'edgar/data/5272/0000950123-02-003222.txt'}

In [132]:
df2.shape

(187869, 5)

In [141]:
#retrieve subset df from index10 and Fraud2 cases 
df6=df2.loc[df2[4].isin(SECFNAME_list)]
#df6.to_csv('FraudCases9319.csv', header=False, index=False)
df6.head()

,0,1,2,3,4
26760,10-K,COMPUTER ASSOCIATES INTERNATIONAL INC,356028,1999-05-28,edgar/data/356028/0000356028-99-000024.txt
27679,10-K,ASTHMA DISEASE MANAGEMENT INC,774746,1999-08-09,edgar/data/774746/0000774746-99-000009.txt
27782,10-K,DCI TELECOMMUNICATIONS INC,769852,1999-07-01,edgar/data/769852/0000769852-99-000018.txt
27798,10-K,DT INDUSTRIES INC,918999,1999-09-27,edgar/data/918999/0001014858-99-000065.txt
28140,10-K,SUPREMA SPECIALTIES INC,872867,1999-09-09,edgar/data/872867/0000891554-99-001776.txt


In [142]:
df6.shape
#255Fraud Cases of the Fraud2 Folder were found in the IndexList1993-12019 of all 10k sec filed 

(255, 5)

In [173]:
len(SECFNAME_list)

403

In [145]:
#secfname List that are mached in 10kindex and in the Fraud2 Folder 
Fraud2_IndexList=df6[4].tolist()
len(Fraud2_IndexList)

255

In [177]:
len(df6[4].tolist())
removeduplicate = list( dict.fromkeys(df6[4].tolist()))
len(removeduplicate)
#No Duplicates in df6.tolist

255

In [178]:
len(SECFNAME_list)

403

In [179]:
removeduplicateSECFNAME_list = list( dict.fromkeys(SECFNAME_list))
len(removeduplicateSECFNAME_list)
#403-371 duplicates in SECFName_list

371

In [181]:
DuplicatesSECFNAME_list=set([x for x in SECFNAME_list if SECFNAME_list.count(x) > 1])

In [182]:
DuplicatesSECFNAME_list

{''}

In [195]:
#List of all secfnames that are in the fraud2 folder but not found/Matched with 10kindex.csv
Fraud2NOTIN10kIndexList=set(SECFNAME_list)-set(df6[4].tolist())
len(Fraud2NOTIN10kIndexList)

116

In [197]:
dfFraud2_2=dfFraud2.loc[dfFraud2["SECFNAME"].isin(Fraud2NOTIN10kIndexList)]

In [199]:
dfFraud2_2.shape
#148=116 plus die 33 empty secfnames

(148, 4)

In [200]:
#this showes that all of the 10k Forms except from one in the Fraud2 Folder was found iin the index10.List
#the other documents are non 10k documents and teh questions is wether they are still to be included in teh Fraud Data Set
#also a question is wether we can include the 10F cases and were the 33 non Forms come from
dfFraud2_2.groupby("FORM").count()

,CIK,CONAME,SECFNAME
FORM,,,
,33,33,33
10-K,1,1,1
10-K405,50,50,50
10KSB,40,40,40
10KSB40,1,1,1
20-F,23,23,23


In [217]:
#Compare Fraud2 10ks with Meiruts 10Ks Fraud cases
#files that are in  Fraud2 Folder that are not found by Meirut/me are 119 cases
#Files that are in Meiruts AND Fraud2 Cases are 136
intersectionFraud2Meirut=set(Fraud2_IndexList)&set(Df3urlList)
additionalFraudList=set(Fraud2_IndexList)-set(Df3urlList)
#files that are in Fraud2 Folder that are not found by Meirut/me are 119 cases
print(len(intersectionFraud2Meirut))
print(len(additionalFraudList))



136
119


In [211]:
removeduplicatesDf3urlList=list(dict.fromkeys(Df3urlList))
len(removeduplicatesDf3urlList)
#no duplicates in Df3urlList

654

In [218]:
df7=df2.loc[df2[4].isin(additionalFraudList)]

In [224]:
df7.head()

,0,1,2,3,4
26760,10-K,COMPUTER ASSOCIATES INTERNATIONAL INC,356028,1999-05-28,edgar/data/356028/0000356028-99-000024.txt
27782,10-K,DCI TELECOMMUNICATIONS INC,769852,1999-07-01,edgar/data/769852/0000769852-99-000018.txt
27798,10-K,DT INDUSTRIES INC,918999,1999-09-27,edgar/data/918999/0001014858-99-000065.txt
28140,10-K,SUPREMA SPECIALTIES INC,872867,1999-09-09,edgar/data/872867/0000891554-99-001776.txt
28308,10-K,BROOKS AUTOMATION INC,933974,1999-12-29,edgar/data/933974/0000950135-99-005746.txt


In [223]:
df7.to_csv('AdditionalFraud.csv', header=False, index=False)

In [ ]:
#add all additional fraud cases from additionalFraudList to Fraud10k FOlder with ed.download10k function 
#results in 763 Fraud Cases 645 are from meirut hence 118 retrieved from 119 Fisettes cases 

In [227]:
dfAllFraudCases=df3.append(df7)
dfAllFraudCases.shape
dfAllFraudCases.to_csv('AllFraudCases.csv', header=False, index=False)

In [233]:
dfAllFraudCases.shape

(773, 5)